论文：[《Theoretical Impediments to Machine Learning With Seven Sparks from the Causal Revolution》](https://arxiv.org/abs/1801.04016)

参考学习地址：
* [大白话谈因果系列文章（一）：因果推断简介及论文介绍](https://zhuanlan.zhihu.com/p/397796913)
* [大白话谈因果系列文章（二）因果效应估计及论文介绍](https://zhuanlan.zhihu.com/p/397974913)
* [大白话谈因果系列文章（三）估计uplift--传统统计学习方法](https://zhuanlan.zhihu.com/p/398643910)
* [*大白话谈因果系列文章（四）估计uplift--深度学习方法](https://zhuanlan.zhihu.com/p/398938743)
* [大白话谈因果系列文章（五）uplift模型评估](https://zhuanlan.zhihu.com/p/399322196)
* [大白话谈因果系列文章（六）连续性因果效应估计 (continuous treatment effect)](https://zhuanlan.zhihu.com/p/451768178)
* [因果推断--uplift model 评估](https://zhuanlan.zhihu.com/p/343747851)
* [因果推断---原理与方法](https://zhuanlan.zhihu.com/p/403098221)
* [因果推断---原理与方法2](https://zhuanlan.zhihu.com/p/427615354)
* [uplift模型评估指标及分析](https://zhuanlan.zhihu.com/p/493811028)
* [因果推断综述及基础方法介绍（一）](https://zhuanlan.zhihu.com/p/258562953)
* [因果推断综述及基础方法介绍（二）](https://zhuanlan.zhihu.com/p/366702128)

代码库：https://github.com/uber/causalml

# 背景
伯努利提出“不充足理由原则”（类似最大墒原则），并在此之上提出“极限定理，揭示了概率与频率之间具有的密切因果关系。自此非确定性的事实研究展开；但**贝叶斯提出的逆概率定理，认为概率现象也是主观信念程度的变化和更新，让概率也失去了客观性**；21世纪，当经典物理大厦建立起来时，“两朵乌云”（莫雷实验和黑体辐射）创立的量子论和相对论，极大改变了人们对确定性理论的衷心，其他学科更是越来越多的不确定性现实出现，统计学创始人高尔顿和学生皮尔逊用**相关关系取代了因果关系**。

目前，大力推广因果革命之一的2011图灵奖得主Judea Pearl，研究中发现**统计相关性并不能取代因果性**(Yule-Simpson Paradox【x与y边缘正相关，在某个变量z的每一水平上负相关】)，无法处理具有共同混杂因子的变量关系，统计数据常因果颠倒（无方向性），造就伪相关，对数据要求也很高(iid，独立不相关)，泛化性，鲁棒性都很差。（以权益为例，为了促进某些不活跃用户的消费，给不活跃用户发了更大的红包，给活跃用户发了更小的红包，把所有特征扔进模型训练，得出了一个奇怪的结论，给用户越大的红包，用户越不买东西）

因果科学是研究因果关系或回答因果问题的学科。现代因果定义为在保证其他因素不变，改变X引起Y的改变，则X为Y的一个原因。
* 物理学定义因果
  * 物理系统演化动力学，基于时间讨论因果
* 哲学定义因果
  * Type Causality：干预主义思想（某原因导致什么结果），由因推果，用于帮助预测
  * Actual Causality：反事实思维（研究若非，若非过去A事件发生，结果事件B可能不发生），关注事物发生的原因

<img style="display: block; margin: 0 auto;" src="../../../assets/images/uplift-1.png" width = "800" height = "400" align=center />

从现实数据中提取出某些变量间的因果关系，主要就是Judea Pearl的因果信息革命。他提出**因果关系之梯**，根据因果问题的可达性，把任务分为递进的3类，association（what is）-> intervention（what if）-> counterfactuals（retrospection）。传统机器学习（关联层）是在问你what is?，即给定属性，问你是什么的概率； 而第二层（干预层）是在问what if？即，如果我对你做了什么，你会怎么样？终极（反事实层）的当然是回答哲学反事实问题，如果我当时那样做了，会怎么样？(唱："想回到过去，试着让故事继续“）这里层层递进，高层可以回答低层的问题，反之则不行，因为不具备充足的信息。Tips：关联和干预区别就是seeing和doing的区别，pearl发明了do算子，既p(y|x) != p(y|do(x))；反事实层是回答“若非“的问题，要求将现实世界与未发生某行为的反事实世界进行比较（假如当时我没那么做，导致这样的结果概率是什么？）。

因果效应估计框架
* D.B Rubin的Rubin Causal Model (RCM)：基于Potential Outcome Framework，设想与观测相悖情况，是一种反事实因果，被称为Experimental causality（但其一般回答干预层的问题）。因果分析步骤主要有 1. 定义问题构建粗粒化因果图 2. Do-Calculus（干预）基于概率计算效应
* 基于Judea Pearl的结构因果模型(SCM)：Pearl提出小图灵测试是实现真正智能的必要条件，并提出因果推理引擎，以假设（图模型）、数据和Query输入，输出Estimand（基于do-calculus判断query是否可识别）、Estimate（概率估计）和Fit Indices（评估）。
* 一般回答反事实问题需要SCM模型：由图模型（表示因果知识）、反事实和干预逻辑（形式化问题）和结构方程（链接因果知识【图模型】和因果问题【反事实和干预逻辑】的语义）组成。一般步骤为 1. abduction（基于现有事实分布【先验】p(u|e)更新图概率p(u)） 2. action（基于结构方程更新x） 3. prediction（预测反事实）

**统计估计的主要困难是数据缺失，如何去除数据产生的偏差（Debias）是核心主题。Pearl提出解决混杂偏差、选择偏差和迁移学习方法（数据本身特点导致）**。反事实基本定律$Y_x(u)=Y_{M_x}(u)$将SCM和RCM联系起来，左边是反事实，$M_x$是干预后的模型。

# 应用
因果推断在互联网界应用主要是基于Uplift model来预测额外收益提升ROI。以营销类业务举例，判定商家决定是否给用户发优惠券促进销售，以buy if treated和buy if not treated组合为四类
* 不给券不买单，给券买单（可被说服的对象）
* 不给券不买单，给券不买单（必不买单）
* 不给券买单，给券买单（必然发生的交易）
* 不给券买单，给券不买单（sleeping dog，反人类）

营销时更多的是想给那些不发券不买单且给券买单的客户（可被说服的对象）来发券，才能最大化实现我们的营销目标。基于业务目标，从哲学角度来看主要是一个type Causality类型的干预主义的预测问题，而非是一个反事实推理问题，所以uplift model是一个干预模型，且为RCM框架。<font color="red">**注意：一般的预测模型是预测这个人给券后的购买概率，这不是我们要的，我们要的是这个人给券比不给券购买概率的增量**</font>

定义效应：
* ITE（Individual Treatment Effect）：$ITE=Y_i(T=1)-Y_i(T=0)$
* ATE（Average Treatment Effect）：$ATE=\mathbb{E}[Y(T=1)-Y(T=0)]$
* CATE（Conditional Average Treatment Effect）：$CATE=\mathbb{E}[Y(T=1)-Y(T=0)|X=x]$

所以目标是个个体给券不给券的区别，而不是总体给券不给券的区别，所以这里就不是研究ATE，而是ITE。由于使用observational data，ITE时CATE的一个特殊情况，所以其实研究的是CATE。

## 目标
Uplift modelling目标精确学习给定一个干预（发券）后，对结果（是否购买概率）增量。即它要建模出增量效应（发券对比不发券对结果的影响），用经济学描述就是学习模型可以最大化边际效用。Uplift定义为：
$$
\begin{align}
u(x) = P(O=1|T;x) - P(O=0|\bar{T};x)
\end{align}
$$
其中$T$为是否发券，$O=1$为买单，$x$为其他特征属性。干预问题和ML的相关问题最直接的区别就是，ML监督模型都是有label的（例如图像识别等，时间和对象不敏感，iid set），而因果干预问题是缺失label的（对于每一个人他只有发券或者不发券的结果【另一个是反事实的】）。建模目标是干预（do操作），而非简单的相关（转化率）。我们希望模型能找到最多的可以被干预转化（发券购买）的用户（我们希望看到用户行为的变化）。

## 实验方法
### 随机试验 $\rightarrow$ CIA
因果推断的黄金准则。其类似RL中核心操作是决策action（do算子）最大化reward（对应uplift），所以uplift model与RL如出一辙。RL中学习过程通过E&E(explore & exploitation)来探索利用action，那uplift model最好也需要EE，即随机试验，因为互联网产品都可以做ABtest，do可以作为ABtest分组，进而进行统计计算。

RCM中$u(x)$定义一个人的uplift，每个个体仅存在被发券或不发券中一种事实，另一种为反事实性存在，所以个体的因果作用不可识别。所以对于个体而言，uplift是确定的，常识别总体（子群体）的因果作用，这里随机性体现在个体$i$上，一般用平均因果作用ACE(Average Causal Effect)表达。
$$
\begin{align}
ACE(Z\rightarrow Y) &= \mathbb{E}(Y_i(1) - Y_i(0)) \\
 &= \mathbb{E}(Y_i(1)|Z_i=1) - \mathbb{E}(Y_i|Z_i=0) \\
 &= \mathbb{E}(Y_i|Z_i=1) - \mathbb{E}(Y_i|Z_i=0) \\
\end{align}
$$
最后一个等式可以利用实验数据进行统计估计，第二个等号成立的核心假设为$Z\perp (Y(1), Y(0))$，所以**随机化实验对于ACE计算起着至关重要的地位**。从子人群（类似根据特征$X$聚类）角度分析ACE，接受发券可能与个人属性$X$相关，破坏上述假设，一般使用<font color="red">**CIA假设（Conditional Independent Assumption）$Z\perp (Y(1), Y(0))｜X$**</font>。也即在$X$每个取值下，希望发券与不发券随机进行。至此可设计ABtest了。

但是，“匹配用户的变量 完全相等” 这个要求过于严格，不实用。由此衍生了用倾向性得分进行匹配的方式Propensity Score Matching，简称 PSM：在 PSM 方法中，我们首先对每一个用户计算一个倾向性得分（propensity score），定义为$e(x)=Pr(T=1|X=x)$，这里的假定由$T\perp (Y(1),Y(0))$变为了$T\perp (Y(1),Y(0))|e(x)$，接着根据倾向性的粉对用户进行匹配。但这种方式也有局限性，就是在于treatment和control用户需要足够齐全且维度较高，想找到倾向性得分相近程度达到要求的客户很难：
* Subclassification：用倾向性得分来对用户分组
* Inverse Propensity Score Weighting（IPSW）：用倾向性得分来对用户进行加权抽样，$w_i=t_i/e(x)+(1-t_i)/(1-e(x))$

### DID (Difference in Differences)
双重差分法**利用趋势的差异计算效应**，主要用于识别ATT（Average Treatment Effect on the treated， $ATT=\mathbb{E}[Y(1)-Y(0)|T=1]$）。引入时间维度$\tau$，$Y_{\tau}(t)$表示treatment t在时间$\tau$的potential outcome，通常假定$\tau=0$时treatment还未授予，$\tau=1$表示treatment已经授予，所以我们关心的量为$ATT=\mathbb{E}[Y_1(1)-Y_1(0)|T=1]$。DID的假设：
* 一致性假设：$\forall \tau, T=t \rightarrow Y_{\tau}(t)$
* 平行趋势假设：若不施加treatment，treatment和control组Y的趋势随时间变化一致。$\mathbb{E}[Y_1(0)-Y_0(0)|T=1]=\mathbb{E}[Y_1(0)-Y_0(0)|T=0]$
* 施加treatment之前无增益，$\mathbb{E}[Y_0(1)-Y_0(0)|T=1]=0$

<img style="display: block; margin: 0 auto;" src="../../../assets/images/uplift-did.png" width = "1000" height = "400" align=center />

parallel trends assumption意味着灰色虚线，说明如果不施加treatment，treatment组内效应增长趋势与control组一致；则我们要识别的ATT就是$\tau=1$施加treatment之后，treatment组效应随时间的增加$\mathbb{E}[Y_1|T=1]-\mathbb{E}[Y_0|T=1]$（红色），减去control组随时间效应的增加$\mathbb{E}[Y_1|T=0]-\mathbb{E}[Y_0|T=0]$（蓝色）


## 建模方法
* Tree-based：和ML的集成树相同，BART，Casual Forest，upliftRF
* Regression-based（meta learning）：首先Estimate the conditional mean outcome E[Y|X = x]（CATE），然后 Derive the CATE estimator based on the difference of results obtained from step 1
  * One-model（S-learner）：把$T$（是否发券）作为特征加入，最终输出$u(x) = f(x,t=1) - f(x,t=0)$，干预组和非干预组一起训练，解决了bias不一致的问题，但是如果本身X的high dimension可能会导致treatment丢失效果
  * Two-model（T-learner/X-learner）：实验对照组分别一个模型，然后做差。可能实验对照各自学的好，但是缺失了我们要的uplift增益效应。优点在于能够很好地区分干预组和非干预组，缺点则在于容易出现两个模型的Bias方向不一致，形成误差累积，使用时需要针对两个模型打分分布做一定校准。X-learner相对于T-learner会对每个样本生成一个反事实预估，于是可以对control组和treatment组分别计算difference，然后再用一个机器学习模型来拟合这两个diffrence，最终对于一个新样本的CATE就是这两个的加权平均。
* 聚类：目标上想圈一部分uplift高的人，且无label指导
* 表示学习（深度模型），本身由于selection bias的存在，导致treament group和control group的人群自带偏差，而类似S-learner的方法又会使得treat的作用丢失，那么将人群embedding中并尽可能消除bias和保存treat的作用就非常重要。代表算法，BNN（balancing neural network）/BLR（balancing linear regression）、TARNet/CFR、ACE、SITE、Perfect Match、Dragonnet、CEVAE等

## 评估
### Uplift by deciles graph
按照预估uplift降序排，按照直方图思路选择一些quantile分组（树模型直接对叶子进行计算排序）。因为我们之前提到RCM是估计"总体“ uplift。再根据每个quantile统计出真实的uplift，用于对比分布差异（uplift by deciles graph）。RCM的experimental causal核心是，假想每一个treat组的人都有一个counterpart在control组，这一点在uplift model是被model默认考虑进的。实际上在因果推断中，对照组和实验组的matching strategy也是一个研究方向，比如kernel matching、propensity score matching（逆概率加权）、 Mahalanobis distance matching等，可以作为优化点。

在评估问题上，传统ML模型需要准确，而uplift model不但需要准确，还要能找出最大化效应的人群。（举个例子，比如模型把所有人uplift预估为整体平均uplift，误差为0，但是模型不知该选谁发券，就没有意义了）。重点关注哪些指标需要根据不同业务具体分析。Radcliffe提出了五点可以作为评估uplift model的指标，大致分为准确性指标和效应指标(模型能否更好把人群分类，ROI指标)。
* Monotonicity（单调性，真实效应高的排在前面，类似皮尔逊相关系数。准确性指标）：predicted uplift越大的segment，actual uplift同样也越大
* Prediction error）（真实与预测uplift误差。准确性指标）：每个segment里，actual uplift和predicted uplift在数值上是否足够接近
* Spread（效应指标）
* Maximum cumulative uplift（累计能获得多少ROI，效应指标）
* Impact at cut-off（效应指标）
准确性和高效应不像ML中偏差和方差，是不矛盾的。但是基于统计的模型，高效应的更容易被异常值影响（均值回归现象），一般任务更倾向于效应指标，毕竟提升ROI才是业务！还有一些业务是目标发现negative effect的（发券更不买的）。

### Uplift Curve
横轴为前$k/n$%的观测样本，纵轴是这些观测样本中treatment组中$Y=1$的个数减去control组中$Y=1$的个数。baseline是起止点的连线，代表平均的样本分布。uplift curve直观解释就是uplift最大的前$k$个样本里，treatment组中$Y=1$的个数比control组中$Y=1$的个数的差值。希望在$k$越小的地方，二者差值越大。这里的差值是一个绝对值的差，这并不合理，如果本身这个测试集treatment组的数量就显著的小于control组，那就算这个uplift模型再好，这个差值可能都是负的。

### AUUC
AUUC就是uplift curve和baseline两条线中间的面积。

### Qini Curve
Qini Curve用于评估后两个指标，类似于gain chart（但纵轴不同），是一种rank-based的累加性指标，即通过降序排列模型打分，绘制总人数（x轴）和截止当前人数中实际购买人数（y轴）。得到Qini Curve只需要把y轴的累计购买人数(cumulative number of purchases)换成增加购买的人数 (incremental number of purchases， 也即y轴是treatmentt中Y=1的个数减去control中Y=1的个数的累加和），此时我们考虑子人群(Segment)，且默认是按照uplift降序排列过。根据每个人群实验对照组购买的累计增量，与圈人数（target number）作图。制作Qini Curve时，因为我们base是依据实验组的target人数，所以在计算incremental时，如果对照和实验组的累计target人数不一样，需要等比例放缩到实验组人数上 $U_k^a=R_k^t-N_k^tR_k^c/N_k^c$

类似gains chart，gini coefficient也可以用作评估。